In [ ]:
def getUserPositiveItem(frame, userID):
    '''
    获取用户正反馈物品：用户评分过的物品
    :param frame: ratings数据
    :param userID: 用户ID
    :return: 正反馈物品
    '''
    series = frame[frame['UserID'] == userID]['MovieID']
    positiveItemList = list(series.values)
    return positiveItemList

def getUserNegativeItem(frame, userID):
    '''
    获取用户负反馈物品：热门但是用户没有进行过评分 与正反馈数量相等
    :param frame: ratings数据
    :param userID:用户ID
    :return: 负反馈物品
    '''
    userItemlist = list(set(frame[frame['UserID'] == userID]['MovieID']))                       
    #用户评分过的物品
    otherItemList = [item for item in set(frame['MovieID'].values) if item not in userItemlist] 
    #用户没有评分的物品
    itemCount = [len(frame[frame['MovieID'] == item]['UserID']) for item in otherItemList]      
    #物品热门程度
    series = pd.Series(itemCount, index=otherItemList)
    series = series.sort_values(ascending=False)[:len(userItemlist)]                            
    #获取正反馈物品数量的负反馈物品
    negativeItemList = list(series.index)
    return negativeItemList
def initPara(userID, itemID, classCount):
    '''
    初始化参数q,p矩阵, 随机
    :param userCount:用户ID
    :param itemCount:物品ID
    :param classCount: 隐类数量
    :return: 参数p,q
    '''
    arrayp = np.random.rand(len(userID), classCount)
    arrayq = np.random.rand(classCount, len(itemID))
    p = pd.DataFrame(arrayp, columns=range(0,classCount), index=userID)
    q = pd.DataFrame(arrayq, columns=itemID, index=range(0,classCount))
    return p,q
def lfmPredict(p, q, userID, itemID):
    '''
    利用参数p,q预测目标用户对目标物品的兴趣度
    :param p: 用户兴趣和隐类的关系
    :param q: 隐类和物品的关系
    :param userID: 目标用户
    :param itemID: 目标物品
    :return: 预测兴趣度
    '''
    p = np.mat(p.ix[userID].values)
    q = np.mat(q[itemID].values).T
    r = (p * q).sum()
    r = sigmod(r)
    return r

def sigmod(x):
    '''
    单位阶跃函数,将兴趣度限定在[0,1]范围内
    :param x: 兴趣度
    :return: 兴趣度
    '''
    y = 1.0/(1+exp(-x))
    return y

def LFM(user_items, F, N, alpha, lambda):  
    #初始化P,Q矩阵  
    [P, Q] = InitModel(user_items, F)  
    #开始迭代  
    For step in range(0, N):  
        #从数据集中依次取出user以及该user喜欢的iterms集  
        for user, items in user_item.iterms():  
            #随机抽样，为user抽取与items数量相当的负样本，并将正负样本合并，用于优化计算  
            samples = RandSelectNegativeSamples(items)  
            #依次获取item和user对该item的兴趣度  
            for item, rui in samples.items():  
                #根据当前参数计算误差  
                eui = eui - Predict(user, item)  
                #优化参数  
                for f in range(0, F):  
                    P[user][f] += alpha * (eui * Q[f][item] - lambda * P[user][f])  
                    Q[f][item] += alpha * (eui * P[user][f] - lambda * Q[f][item])  
        #每次迭代完后，都要降低学习速率。一开始的时候由于离最优值相差甚远，因此快速下降；  
        #当优化到一定程度后，就需要放慢学习速率，慢慢的接近最优值。  
        alpha *= 0.9  
        
def recommend(frame, userID, p, q, TopN=10):
    '''
    推荐TopN个物品给目标用户
    :param frame: 源数据
    :param userID: 目标用户
    :param p: 用户兴趣和隐类的关系
    :param q: 隐类和物品的关系
    :param TopN: 推荐数量
    :return: 推荐物品
    '''
    userItemlist = list(set(frame[frame['UserID'] == userID]['MovieID']))
    otherItemList = [item for item in set(frame['MovieID'].values) if item not in userItemlist]
    predictList = [lfmPredict(p, q, userID, itemID) for itemID in otherItemList]
    series = pd.Series(predictList, index=otherItemList)
    series = series.sort_values(ascending=False)[:TopN]
    return series


In [23]:
# coding:utf-8
__author__ = "orisun"
 
import random
import math
 
class LFM(object):
 
    def __init__(self, rating_data, F, alpha=0.1, lmbd=0.1, max_iter=500):
        '''rating_data是list<(user,list<(position,rate)>)>类型
        '''
        self.F = F
        self.P = dict()  # R=PQ^T，代码中的Q相当于博客中Q的转置
        self.Q = dict()
        self.alpha = alpha
        self.lmbd = lmbd
        self.max_iter = max_iter
        self.rating_data = rating_data
 
        '''随机初始化矩阵P和Q'''
        for user, rates in self.rating_data:
            self.P[user] = [random.random() / math.sqrt(self.F)
                            for x in xrange(self.F)]
            for item, _ in rates:
                if item not in self.Q:
                    self.Q[item] = [random.random() / math.sqrt(self.F)
                                    for x in xrange(self.F)]
 
    def train(self):
        '''随机梯度下降法训练参数P和Q
        '''
        for step in xrange(self.max_iter):
            for user, rates in self.rating_data:
                for item, rui in rates:
                    hat_rui = self.predict(user, item)
                    err_ui = rui - hat_rui
                    for f in xrange(self.F):
                        self.P[user][f] += self.alpha * (err_ui * self.Q[item][f] - self.lmbd * self.P[user][f])
                        self.Q[item][f] += self.alpha * (err_ui * self.P[user][f] - self.lmbd * self.Q[item][f])
            self.alpha *= 0.9  # 每次迭代步长要逐步缩小
 
    def predict(self, user, item):
        '''预测用户user对物品item的评分
        '''
        return sum(self.P[user][f] * self.Q[item][f] for f in xrange(self.F))

if __name__ == '__main__':
    '''用户有A B C，物品有a b c d'''
    rating_data = list()
    rate_A = [('a', 1.0), ('b', 1.0)]
    rating_data.append(('A', rate_A))
    rate_B = [('b', 1.0), ('c', 1.0)]
    rating_data.append(('B', rate_B))
    rate_C = [('c', 1.0), ('d', 1.0)]
    rating_data.append(('C', rate_C))
    #sample movie examples
#     rate_A = [('Harry Potter',1.0),('Avatar',1.0),('LOTR 3',1.0),('Gladiator',0.0),('Titanic',0.0),('Glitter',0.0)]
#     rating_data.append(('Alice',rate_A))
#     rate_B = [('Harry Potter',1.0),('Avatar',0.0),('LOTR 3',1.0),('Gladiator',0.0),('Titanic',0.0),('Glitter',0.0)]
#     rating_data.append(('Bob',rate_B))#SF/fantasy fan
#     rate_C = [('Harry Potter',1.0),('Avatar',1.0),('LOTR 3',1.0),('Gladiator',0.0),('Titanic',0.0),('Glitter',0.0)]
#     rating_data.append(('Carol',rate_C))#Big SF/fantasy fan
#     rate_D = [('Harry Potter',0.0),('Avatar',0.0),('LOTR 3',1.0),('Gladiator',1.0),('Titanic',1.0),('Glitter',0.0)]
#     rating_data.append(('David',rate_D))#Big Oscar winners fan
#     rate_E = [('Harry Potter',0.0),('Avatar',0.0),('LOTR 3',1.0),('Gladiator',1.0),('Titanic',1.0),('Glitter',0.0)]
#     rating_data.append(('Eric',rate_E))#Oscar winners fan,except for Titanic
#     rate_F = [('Harry Potter',0.0),('Avatar',0.0),('LOTR 3',1.0),('Gladiator',1.0),('Titanic',1.0),('Glitter',0.0)]
#     rating_data.append(('Fred',rate_F))#Oscar winners fan,except for Titanic
#     rate_G = [('Harry Potter',0), ('Avatar',0),('LOTR 3',0), ('Gladiator',1.0), ('Titanic',1.0), ('Glitter',0)] 
    lfm = LFM(rating_data, 2)
    lfm.train()
    for item in ['a', 'b', 'c', 'd']:
        print item, lfm.predict('C', item)      #计算用户A对各个物品的喜好程度
#     for item in ['Harry Potter','Avatar','LOTR 3','Gladiator','Titanic','Glitter']:
#         print item, lfm.predict('Alice',item)
#     for u in sorted(lfm.P.keys()):
#         print u,lfm.P[u]


a 0.809217334785
b 1.01209277051
c 0.894069356814
d 0.694955681096


In [21]:
# coding=utf-8
__author__ = "yanbin"
 
import numpy as np
 
class RBM(object):
 
    def __init__(self, num_visible, num_hidden, learn_rate=0.1, learn_batch=1000):
        self.num_visible = num_visible  # 可视层神经元个数
        self.num_hidden = num_hidden    # 隐藏层神经元个数
        self.learn_rate = learn_rate    # 学习率
        self.learn_batch = learn_batch  # 每次根据多少样本进行学习
 
        '''初始化连接权重'''
        self.weights = 0.1 * \
            np.random.randn(self.num_visible,
                            self.num_hidden)  # 依据0.1倍的标准正太分布随机生成权重
        # 第一行插入全0，即偏置和隐藏层的权重初始化为0
        self.weights = np.insert(self.weights, 0, 0, axis=0)
        # 第一列插入全0，即偏置和可视层的权重初始化为0
        self.weights = np.insert(self.weights, 0, 0, axis=1)
 
    def _logistic(self, x):
        '''直接使用1.0 / (1.0 + np.exp(-x))容易发警告“RuntimeWarning: overflowencountered in exp”，
           转换成如下等价形式后算法会更稳定
        '''
        return 0.5 * (1 + np.tanh(0.5 * x))
 
    def train(self, rating_data, max_steps=1000, eps=1.0e-4):
        '''迭代训练，得到连接权重
        '''
        for step in xrange(max_steps):  # 迭代训练多少次
            error = 0.0  # 误差平方和
            # 每次拿一批样本还调整权重
            for i in xrange(0, rating_data.shape[0], self.learn_batch):
                num_examples = min(self.learn_batch, rating_data.shape[0] - i)
                data = rating_data[i:i + num_examples, :]
                data = np.insert(data, 0, 1, axis=1)  # 第一列插入全1，即偏置的值初始化为1
 
                pos_hidden_activations = np.dot(data, self.weights)
                pos_hidden_probs = self._logistic(pos_hidden_activations)
                pos_hidden_states = pos_hidden_probs > np.random.rand(
                    num_examples, self.num_hidden + 1)
                # pos_associations=np.dot(data.T,pos_hidden_states)         #对隐藏层作二值化
                pos_associations = np.dot(
                    data.T, pos_hidden_probs)  # 对隐藏层不作二值化
 
                neg_visible_activations = np.dot(
                    pos_hidden_states, self.weights.T)
                neg_visible_probs = self._logistic(neg_visible_activations)
                neg_visible_probs[:, 0] = 1  # 强行把偏置的值重置为1
                neg_hidden_activations = np.dot(
                    neg_visible_probs, self.weights)
                neg_hidden_probs = self._logistic(neg_hidden_activations)
                # neg_hidden_states=neg_hidden_probs>np.random.rand(num_examples,self.num_hidden+1)
                # neg_associations=np.dot(neg_visible_probs.T,neg_hidden_states)      #对隐藏层作二值化
                neg_associations = np.dot(
                    neg_visible_probs.T, neg_hidden_probs)  # 对隐藏层不作二值化
 
                # 更新权重。另外一种尝试是带冲量的梯度下降，即本次前进的方向是本次梯度与上一次梯度的线性加权和（这样的话需要额外保存上一次的梯度）
                self.weights += self.learn_rate * \
                    (pos_associations - neg_associations) / num_examples
 
                # 计算误差平方和
                error += np.sum((data - neg_visible_probs)**2)
            if error < eps:  # 所有样本的误差平方和低于阈值于终止迭代
                break
            print 'iteration %d, error is %f' % (step, error)
 
    def getHidden(self, visible_data):
        '''根据输入层得到隐藏层
           visible_data是一个matrix，每行代表一个样本
        '''
        num_examples = visible_data.shape[0]
        hidden_states = np.ones((num_examples, self.num_hidden + 1))
        visible_data = np.insert(visible_data, 0, 1, axis=1)  # 第一列插入偏置
        hidden_activations = np.dot(visible_data, self.weights)
        hidden_probs = self._logistic(hidden_activations)
        hidden_states[:, :] = hidden_probs > np.random.rand(
            num_examples, self.num_hidden + 1)
        hidden_states = hidden_states[:, 1:]            # 即首列删掉，即把偏置去掉
        return hidden_states
 
    def getVisible(self, hidden_data):
        '''根据隐藏层得到输入层
           hidden_data是一个matrix，每行代表一个样本
        '''
        num_examples = hidden_data.shape[0]
        visible_states = np.ones((num_examples, self.num_visible + 1))
        hidden_data = np.insert(hidden_data, 0, 1, axis=1)
        visible_activations = np.dot(hidden_data, self.weights.T)
        visible_probs = self._logistic(visible_activations)
        visible_states[:, :] = visible_probs > np.random.rand(
            num_examples, self.num_visible + 1)
        visible_states = visible_states[:, 1:]
        return visible_states
 
    def predict(self, visible_data):
        num_examples = visible_data.shape[0]
        hidden_states = np.ones((num_examples, self.num_hidden + 1))
        visible_data = np.insert(visible_data, 0, 1, axis=1)  # 第一列插入偏置
        '''forward'''
        hidden_activations = np.dot(visible_data, self.weights)
        hidden_probs = self._logistic(hidden_activations)
        # hidden_states[:, :] = hidden_probs > np.random.rand(
        #     num_examples, self.num_hidden + 1)
        '''backward'''
        visible_states = np.ones((num_examples, self.num_visible + 1))
        # visible_activations = np.dot(hidden_states, self.weights.T)  #对隐藏层作二值化
        visible_activations = np.dot(hidden_probs, self.weights.T)  # 对隐藏层不作二值化
        visible_probs = self._logistic(visible_activations)  # 直接返回可视层的概率值
 
        return visible_probs[:, 1:]  # 把第0列(偏置)去掉
 
if __name__ == '__main__':
    rbm = RBM(num_visible=4, num_hidden=2, learn_rate=0.1, learn_batch=1000)
    rating_data = np.array([[1, 1, 1, 0, 0, 0], [1, 0, 1, 0, 0, 0], [1, 1, 1, 0, 0, 0], [
                           0, 0, 1, 1, 1, 0], [0, 0, 1, 1, 0, 0], [0, 0, 1, 1, 1, 0]])
    rating_data = np.array([[1,1,0,0],[1,0,1,0],[0,0,1,1]])
    rbm.train(rating_data,max_steps=500, eps=1.0e-4)
    print 'weight:\n', rbm.weights
    rating = np.array([[0, 0, 0, 0.9, 0.7, 0]])  # 评分需要做归一化。该用户喜欢第四、五项
    rating = np.array([[1,1,0,0]])
    hidden_data = rbm.getHidden(rating)
    print 'hidden_data:\n', hidden_data
    visible_data = rbm.getVisible(hidden_data)
    print 'visible_data:\n', visible_data
    predict_data = rbm.predict(rating)
    print '推荐得分:'
    for i, score in enumerate(predict_data[0, :]):
        print i, score  # 第三、四、五项的推荐得分很高，同时用户已明确表示过喜欢四、五，所以我们把第三项推荐给用户


iteration 0, error is 3.012735
iteration 1, error is 2.995830
iteration 2, error is 2.975186
iteration 3, error is 2.989292
iteration 4, error is 2.928172
iteration 5, error is 3.019785
iteration 6, error is 2.914794
iteration 7, error is 2.859600
iteration 8, error is 2.964442
iteration 9, error is 2.965668
iteration 10, error is 2.964141
iteration 11, error is 2.921345
iteration 12, error is 2.924412
iteration 13, error is 2.888505
iteration 14, error is 3.043583
iteration 15, error is 2.970955
iteration 16, error is 2.858787
iteration 17, error is 2.828750
iteration 18, error is 2.870662
iteration 19, error is 2.829418
iteration 20, error is 2.698671
iteration 21, error is 2.769905
iteration 22, error is 3.027798
iteration 23, error is 2.585815
iteration 24, error is 2.997120
iteration 25, error is 2.858570
iteration 26, error is 2.996520
iteration 27, error is 3.065192
iteration 28, error is 2.617551
iteration 29, error is 2.587556
iteration 30, error is 2.580958
iteration 31, erro